# Accessing ACRIS Records via NYC OpenData and Socrata

In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import json
from datetime import date
import os
from tqdm import tqdm
from sodapy import Socrata

In [2]:
client = Socrata('data.cityofnewyork.us', None)

## 1. ACRIS - Real Property Master Table

The first step is to get data from the master table ([link](https://data.cityofnewyork.us/City-Government/ACRIS-Real-Property-Master/bnx9-e6tj))

### 1.A.I. Get a Total Count of Records

In [3]:
url = 'https://data.cityofnewyork.us/resource/bnx9-e6tj.json?$select=count(*) as row_count'

In [4]:
total_rows = int(requests.get(url).json()[0]['row_count'])
total_rows

15453923

### 1.A.II. Get a Total Count of Records That Are Within Defined Parameters

We want all **deeds** with a **document date** within a given number of years from today, where the **document amount** is greater than zero, and where the **entire parcel** transferred.

In [5]:
# year_range_legnth = 2
# start_year = date.today().year - year_range_legnth
# start_date_string = f"{start_year}-{date.today().month}-{date.today().day}"

#Code for a start date
start_year = 2016
start_month = 1
start_day = 1
start_date_string = f"{start_year}-{start_month}-{start_day}"
start_date_string

'2016-1-1'

In [6]:
url = f"https://data.cityofnewyork.us/resource/bnx9-e6tj.json?doc_type='DEED'&$where=(document_date>'{start_date_string}' AND document_amt>'0')&percent_trans='100'&$select=count(*) as row_count"
total_rows = int(requests.get(url).json()[0]['row_count'])
total_rows

195358

### 1.B. Functions to Access Data From the API

There is a `5000` record limit per request.

We want to pull all **deed** data in manhattan where the entire property transferred and within a specific time frame.

In [7]:
complete_list_of_columns = ['document_id', 'record_type', 'crfn', 'recorded_borough', 'doc_type', 'ASST',
                            'document_date', 'document_amt', 'recorded_datetime', 'modified_date', 'reel_yr',
                           'reel_nbr', 'reel_pg', 'percent_trans', 'good_through_date']

In [8]:
def pull_one_portion_of_data(limit_num, offset_num):
    return(pd.DataFrame.from_records(client.get('bnx9-e6tj', where=f"doc_type='DEED' AND document_date>'{start_date_string}' AND document_amt > '0' AND percent_trans='100'", limit=limit_num, offset=offset_num)))

In [9]:
# Note: If the data set is large you can pull and upload the data one row at a time to our
# database when it is set up

def obtain_table_of_data(limit_num, offset_num):
    
    # Not all columns will be present if the column is blank for all records in the portion
    # This will create an error when you try to merge tables that are missing data
    
    # Pull one portion of data
    table_portion = pull_one_portion_of_data(limit_num, offset_num)
    
    # The columns in the portion of data pulled from the data frame
    df_portion_columns = [x.lower() for x in table_portion.columns]
    
    # A list of missing columns for the portion
    missing_cols = [x for x in complete_list_of_columns if x.lower() not in df_portion_columns]
    
    # Set the values of this column to None
    for i in missing_cols:
        table_portion[i] = None
    
    # You can either return the data portion or upload it to a database
    return table_portion

### 1.C. Obtain the Data within a Specific Time and where the Entire Property Transferred

In [10]:
total_iterations = int(total_rows/5000) + 1
print(f'There are {str(total_iterations)} total number of iterations required to pull the data.')

There are 40 total number of iterations required to pull the data.


In [11]:
def access_all_data(limit_num, starting_iteration):
    
    total_iterations = int(total_rows/5000) + 1
    
    list_of_data_frames = []
    
    for i in tqdm(range(total_iterations - starting_iteration)):
        offset_num = (starting_iteration + i) * limit_num
        
        list_of_data_frames.append(obtain_table_of_data(limit_num, offset_num))
    
    return list_of_data_frames

In [12]:
acris_master_df = access_all_data(limit_num = 5000, starting_iteration = 0)

100%|██████████| 40/40 [01:12<00:00,  1.82s/it]


In [13]:
len(acris_master_df)

40

In [14]:
acris_master_df = pd.concat(acris_master_df)

#acris_master_df.to_csv('acris_master_data_2.csv', index = False)

In [15]:
acris_master_df.head()

,document_id,record_type,crfn,recorded_borough,doc_type,document_date,document_amt,recorded_datetime,modified_date,reel_yr,reel_nbr,reel_pg,percent_trans,good_through_date,ASST
0,2017011101164001,A,2017000019549,1,DEED,2017-01-09T00:00:00.000,435000,2017-01-13T00:00:00.000,2017-01-13T00:00:00.000,0,0,0,100,2017-01-31T00:00:00.000,None
1,2017012100135001,A,2017000029745,1,DEED,2016-11-16T00:00:00.000,400000,2017-01-23T00:00:00.000,2017-01-23T00:00:00.000,0,0,0,100,2017-01-31T00:00:00.000,None
2,2017012401183001,A,2017000036907,1,DEED,2016-12-29T00:00:00.000,975000,2017-01-26T00:00:00.000,2017-01-26T00:00:00.000,0,0,0,100,2017-01-31T00:00:00.000,None
3,2017010900623002,A,2017000030067,3,DEED,2017-01-05T00:00:00.000,370000,2017-01-23T00:00:00.000,2017-01-23T00:00:00.000,0,0,0,100,2017-01-31T00:00:00.000,None
4,2017011701192001,A,2017000024154,1,DEED,2017-01-11T00:00:00.000,515000,2017-01-18T00:00:00.000,2017-01-18T00:00:00.000,0,0,0,100,2017-01-31T00:00:00.000,None


In [16]:
acris_master_df.shape[0]

195358

In [17]:
pd.DataFrame.from_records(client.get('8h5j-fqxa', where=f"document_id='2014100800139003'"))

,document_id,record_type,borough,block,lot,easement,partial_lot,air_rights,subterranean_rights,property_type,street_number,street_name,good_through_date
0,2014100800139003,L,1,573,53,N,E,N,N,F1,17,WEST 9TH STREET,2015-07-31T00:00:00.000


## 2. ACRIS - Real Property Legals

The next step is to get the legal data which can be combined with the masters data ([link](https://data.cityofnewyork.us/City-Government/ACRIS-Real-Property-Legals/8h5j-fqxa)).

We also want to get comparable properties within a specific range of blocks around the target property ([see tax map](http://gis.nyc.gov/taxmap/map.htm))

### 2.A.I. Get a Total Count of Records

In [18]:
url = 'https://data.cityofnewyork.us/resource/8h5j-fqxa.json?$select=count(*) as row_count'

In [19]:
total_rows = int(requests.get(url).json()[0]['row_count'])
total_rows

20359086

### 2.A.II. Get a Total Count of Records That Are Within Defined Parameters

We want all documents with a **property type** of *AP* and where the block falls within the range defined below.

([Property Type Descriptions](https://data.cityofnewyork.us/City-Government/ACRIS-Property-Types-Codes/94g4-w6xz) and [here](https://data.cityofnewyork.us/widgets/94g4-w6xz)) 

I identified a range of blocks in Brooklyn that could be used as a boundary for comparable properties ([see tax map](http://gis.nyc.gov/taxmap/map.htm)):
                    
- Range Description:

    -South of Avenue I
    
    -East of McDonald Avenue and Shell Road
    
    -North of Shore Parkway
    
    -West of Nostrand Avenue


- Corresponding Blocks:
                        1669 East 19th Street, Brooklyn:
                        
                        (block BETWEEN '6514' AND '6546') OR (block BETWEEN '6564' AND '6573') \
                        OR (block BETWEEN '6583' AND '6592') OR (block BETWEEN '6608' AND '6617') \
                        OR (block BETWEEN '6633' AND '6642') OR (block BETWEEN '6658' AND '6666') \
                        OR (block BETWEEN '6681' AND '6685') OR (block BETWEEN '7104' AND '7112') \
                        OR (block BETWEEN '7088' AND '7091') OR (block BETWEEN '7124' AND '7136') \
                        OR (block BETWEEN '7150' AND '7159') OR (block BETWEEN '7173' AND '7184') \
                        OR (block BETWEEN '7192' AND '7204') OR (block BETWEEN '7212' AND '7224') \
                        OR (block BETWEEN '7232' AND '7244') OR (block BETWEEN '6704' AND '6835') \
                        OR (block BETWEEN '7584' AND '7593') OR (block BETWEEN '7602' AND '7611') \
                        OR (block BETWEEN '7620' AND '7629') OR (block BETWEEN '7638' AND '7647') \
                        OR (block BETWEEN '7656' AND '7665') OR (block BETWEEN '7674' AND '7690') \
                        OR (block BETWEEN '7290' AND '7308') OR (block BETWEEN '7315' AND '7334') \
                        OR (block BETWEEN '7343' AND '7362') OR (block BETWEEN '7371' AND '7386') \
                        OR (block BETWEEN '7394' AND '7405') OR (block BETWEEN '7411' AND '7422') \
                        OR (block BETWEEN '7430' AND '7445') OR (block BETWEEN '7454' AND '7474') \
                        OR (block BETWEEN '8768' AND '8791') OR (block = '5495')

A visualization of the range:

![title](Notebook_Images/Greenwich_Avenue_Comparable_Boundaries.png)
*Note: The star represents the location of the portfolio*

In [20]:
comparable_blocks = "(block BETWEEN '6514' AND '6546') OR (block BETWEEN '6564' AND '6573') \
                  OR (block BETWEEN '6583' AND '6592') OR (block BETWEEN '6608' AND '6617') \
                  OR (block BETWEEN '6633' AND '6642') OR (block BETWEEN '6658' AND '6666') \
                  OR (block BETWEEN '6681' AND '6685') OR (block BETWEEN '7104' AND '7112') \
                  OR (block BETWEEN '7088' AND '7091') OR (block BETWEEN '7124' AND '7136') \
                  OR (block BETWEEN '7150' AND '7159') OR (block BETWEEN '7173' AND '7184') \
                  OR (block BETWEEN '7192' AND '7204') OR (block BETWEEN '7212' AND '7224') \
                  OR (block BETWEEN '7232' AND '7244') OR (block BETWEEN '6704' AND '6835') \
                  OR (block BETWEEN '7584' AND '7593') OR (block BETWEEN '7602' AND '7611') \
                  OR (block BETWEEN '7620' AND '7629') OR (block BETWEEN '7638' AND '7647') \
                  OR (block BETWEEN '7656' AND '7665') OR (block BETWEEN '7674' AND '7690') \
                  OR (block BETWEEN '7290' AND '7308') OR (block BETWEEN '7315' AND '7334') \
                  OR (block BETWEEN '7343' AND '7362') OR (block BETWEEN '7371' AND '7386') \
                  OR (block BETWEEN '7394' AND '7405') OR (block BETWEEN '7411' AND '7422') \
                  OR (block BETWEEN '7430' AND '7445') OR (block BETWEEN '7454' AND '7474') \
                  OR (block BETWEEN '8768' AND '8791') OR (block = '5495')"

target_borough = '3'
target_block   = '6782'
target_lot     = '59'
#target_lot = '3'

In [21]:
# url = f'''https://data.cityofnewyork.us/resource/8h5j-fqxa.json?borough='{target_borough}'&$where=({comparable_blocks}
#                                                                 )&property_type='MC'&$select=count(*) as row_count'''

url = f'''https://data.cityofnewyork.us/resource/8h5j-fqxa.json?borough='{target_borough}'&$where=((property_type='AP') AND ({comparable_blocks}
                                                                ))&$select=count(*) as row_count'''


total_rows = int(requests.get(url).json()[0]['row_count'])
total_rows

8892

### 2.B. Functions to Access Data From the API

In [22]:
complete_list_of_columns = ['document_id', 'record_type', 'borough', 'block', 'lot', 'easement', 'partial_lot',
                            'air_rights', 'subterranean_rights', 'property_type', 'street_number',
                            'street_name', 'unit', 'good_through_date']

In [23]:
def pull_one_portion_of_data(limit_num, offset_num, comparable_blocks, target_borough):
    return(pd.DataFrame.from_records(client.get('8h5j-fqxa', where=f'''borough='{target_borough}' AND 
                                                                                  ({comparable_blocks}
                                                                                )
                                                                                AND (property_type='AP')
                                                ''', limit=limit_num, offset=offset_num)))

In [24]:
# Note: If the data set is large you can pull and upload the data one row at a time to our
# database when it is set up

def obtain_table_of_data(limit_num, offset_num, comparable_blocks, target_borough):
    
    # Not all columns will be present if the column is blank for all records in the portion
    # This will create an error when you try to merge tables that are missing data
    
    # Pull one portion of data
    table_portion = pull_one_portion_of_data(limit_num, offset_num, comparable_blocks, target_borough)
    
    # The columns in the portion of data pulled from the data frame
    df_portion_columns = [x.lower() for x in table_portion.columns]
    
    # A list of missing columns for the portion
    missing_cols = [x for x in complete_list_of_columns if x.lower() not in df_portion_columns]
    
    # Set the values of this column to None
    for i in missing_cols:
        table_portion[i] = None
    
    # You can either return the data portion or upload it to a database
    return table_portion

### 2.C. Obtain the Data

In [25]:
total_iterations = int(total_rows/5000) + 1
print(f'There are {str(total_iterations)} total number of iterations required to pull the data.')

There are 2 total number of iterations required to pull the data.


In [26]:
def access_all_data(limit_num, starting_iteration, comparable_blocks, target_borough):
    
    total_iterations = int(total_rows/5000) + 1
    
    list_of_data_frames = []
    
    for i in tqdm(range(total_iterations - starting_iteration)):
        offset_num = (starting_iteration + i) * limit_num
        
        list_of_data_frames.append(obtain_table_of_data(limit_num, offset_num, comparable_blocks, target_borough))
    
    return list_of_data_frames

In [27]:
acris_legals_df = access_all_data(limit_num = 5000, starting_iteration = 0, comparable_blocks = comparable_blocks, target_borough = target_borough)

100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


In [28]:
len(acris_legals_df)

2

In [29]:
acris_legals_df = pd.concat(acris_legals_df)
#acris_legals_df.to_csv('acris_legals_data.csv', index = False)

In [30]:
acris_legals_df.head()

,document_id,record_type,borough,block,lot,easement,partial_lot,air_rights,subterranean_rights,property_type,street_number,street_name,good_through_date,unit
0,2015081001601001,L,3,6805,66,N,E,N,N,AP,1803,EAST 22ND STREET,2015-08-31T00:00:00.000,NaN
1,2015082000111002,L,3,6712,86,N,E,N,N,AP,1464,OCEAN AVENUE,2015-08-31T00:00:00.000,NaN
2,2015081100176005,L,3,7405,52,N,E,N,N,AP,2765-2777,OCEAN AVENUE,2015-08-31T00:00:00.000,NaN
3,2015082000111001,L,3,6712,86,N,E,N,N,AP,1468,OCEAN AVENUE,2015-08-31T00:00:00.000,NaN
4,2015072901059001,L,3,6531,74,N,E,N,N,AP,1021,EAST 10TH STREET,2015-08-31T00:00:00.000,NaN


In [31]:
acris_legals_df.shape

(8892, 14)

## 3. Find Data where the records match

In [32]:
df = pd.merge(acris_master_df, acris_legals_df, on=['document_id','document_id'], how='left', indicator='Exist').copy()

In [101]:
comparables_df = df[["document_id", "document_date", "borough", "block", "lot", "document_amt", "street_number", "street_name", "unit"]][df['Exist']=='both'].copy()
#comparables_df.to_csv('comparable_sales.csv', index = False)

In [102]:
print(comparables_df.shape[0])

53


## 4. Property Valuation and Assessment Data Tax Classes 1,2,3,4

Source: https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html

### 4.A. A function to get the data for a specific property

In [103]:
vdf_data_frame_columns = ["boro", "block", "lot", "year", "extracrdt", "bldg_class", "owner", "zoning", "num_bldgs", "yrbuilt", "period", "land_area", "gross_sqft", "office_area_gross", "residential_area_gross", "retail_area_gross", "units"]

In [104]:
def pull_one_property_data(boro, block, lot):
    purl = f"https://data.cityofnewyork.us/resource/8y4t-faws.json?boro='{boro}'&block='{block}'&lot='{lot}'"
    result_rows = requests.get(purl).json()
    results_df = pd.DataFrame(result_rows).sort_values(by = "extracrdt", ascending = False)[[#"boro","block", "lot", "aptno"
                                                                                             "year", "extracrdt", "bldg_class",
                                                                                             "owner", "num_bldgs", "yrbuilt",
                                                                                             "period", "bld_story", "gross_sqft",
                                                                                             "office_area_gross", "residential_area_gross",
                                                                                             "retail_area_gross", "units"]].iloc[0]
    
    
    #if results_df.shape[0] == 0:
    #    return results_df

    return results_df
    

In [105]:
# pull_one_property_data(target_borough, target_block, target_lot)
# pull_one_property_data(target_borough, "636", "1204")  #.to_csv("test.csv")
target_bldg_class = pull_one_property_data(target_borough, target_block, target_lot)['bldg_class']

### 4.B. A Function to Get the Data for a Property within a Dataframe

In [106]:
def pull_one_property_data(x):
    purl = f"https://data.cityofnewyork.us/resource/8y4t-faws.json?boro='{x['borough']}'&block='{x['block']}'&lot='{x['lot']}'"
    result_rows = requests.get(purl).json()
    results_df = pd.DataFrame(result_rows).sort_values(by = "extracrdt", ascending = False)
    
    
    # The columns in the portion of data pulled from the data frame
    df_portion_columns = [x.lower() for x in results_df.columns]
    
    complete_list_of_columns = ["year", "extracrdt", "bldg_class","owner", "num_bldgs", "yrbuilt", "yralt1", "yralt2",
                                "period", "bld_story", "gross_sqft", "office_area_gross", "residential_area_gross",
                                "retail_area_gross", "hotel_area_gross", "loft_area_gross", "factory_area_gross", 
                                "warehouse_area_gross", "storage_area_gross", "garage_area", "other_area_gross",
                                "units"]
    
    
    # A list of missing columns for the portion
    missing_cols = [x for x in complete_list_of_columns if x.lower() not in df_portion_columns]
    
    # Set the values of this column to None
    for i in missing_cols:
        results_df[i] = None
    
    
    results_df = results_df[[#"boro","block", "lot",
                             "year", "extracrdt", "bldg_class",
                             "owner", "num_bldgs", "yrbuilt",
                             "yralt1", "yralt2",
                             "period", "land_area", "bld_story", "gross_sqft",
                             "lot_frt", "bld_frt", "lot_dep", "bld_dep",
                             "office_area_gross", "residential_area_gross",
                             "retail_area_gross", 
                             "hotel_area_gross", "loft_area_gross",
                             "factory_area_gross", "warehouse_area_gross",
                             "storage_area_gross", "garage_area", "other_area_gross",
                              "units"]].iloc[0]
    
    
    #if results_df.shape[0] == 0:
    #    return results_df

    return results_df
    

In [107]:
# purl = f"https://data.cityofnewyork.us/resource/8y4t-faws.json?boro='1'&block='616'&lot='1105'"
# result_rows = requests.get(purl).json()
# pd.DataFrame(result_rows)

In [108]:
# comparables_df.iloc[104]

In [109]:
comparables_df = comparables_df.drop(comparables_df.index[[4,5,8,9,25,26]])

In [110]:
pull_one_property_data(comparables_df.iloc[39])

year                                         2022
extracrdt                 2021-05-24T00:00:00.000
bldg_class                                     C1
owner                            NOSTRAND EG, LLC
num_bldgs                                       1
yrbuilt                                      1954
yralt1                                          0
yralt2                                          0
period                                          3
land_area                                   36000
bld_story                                       3
gross_sqft                                  47000
lot_frt                                       320
bld_frt                                       240
lot_dep                                       110
bld_dep                                        75
office_area_gross                               0
residential_area_gross                      47000
retail_area_gross                               0
hotel_area_gross                                0


### 4.C. A Function to Get the Pad Data for a Given Property in the Comparables Table

In [111]:
tqdm.pandas()

comparables_df[["year",
              "extracrdt",
              "bldg_class",
              "owner",
#               "zoning",
              "num_bldgs",
              "yrbuilt",
              "yralt1",
              "yralt2",
              "period",
              "land_area",
              "bld_story",
              "gross_sqft",
              "lot_frt",
              "bld_frt",
              "lot_dep",
              "bld_dep",
              "office_area_gross",
              "residential_area_gross",
              "retail_area_gross",
              "hotel_area_gross", "loft_area_gross",
              "factory_area_gross", "warehouse_area_gross",
              "storage_area_gross", "garage_area", "other_area_gross",
              "units"]] = comparables_df.progress_apply(
    lambda row: pull_one_property_data(row), axis=1)



100%|██████████| 47/47 [00:12<00:00,  3.77it/s]


In [112]:
#comparables_df['document_id_test'] = comparables_df['document_id'].astype('int')
comparables_df['document_id'] = "Doc " + comparables_df['document_id']

In [113]:
# A count of how many transacted buildings have the same building class as the target property
comparables_df[comparables_df['bldg_class'] == target_bldg_class]

,document_id,document_date,borough,block,lot,document_amt,street_number,street_name,unit,year,...,residential_area_gross,retail_area_gross,hotel_area_gross,loft_area_gross,factory_area_gross,warehouse_area_gross,storage_area_gross,garage_area,other_area_gross,units
2025,Doc 2017012600044001,2017-01-24T00:00:00.000,3,7397,34,37000000,1245,AVENUE X,135,2022,...,124636,0,0,0,0,0,0,0,0,136
3022,Doc 2017020900851001,2017-02-08T00:00:00.000,3,7380,34,13000000,2686,OCEAN AVENUE,NaN,2022,...,54000,0,0,0,0,0,0,0,0,48
5431,Doc 2017030100438001,2017-02-16T00:00:00.000,3,6541,46,4100000,1296,OCEAN PARKWAY,NaN,2022,...,9100,0,0,0,0,0,0,0,0,7
45796,Doc 2018020500568001,2018-02-01T00:00:00.000,3,6803,80,10000000,2197,OCEAN AVE,NaN,2022,...,30000,0,0,0,0,0,0,0,0,35
56498,Doc 2018061400620004,2018-06-06T00:00:00.000,3,7194,28,15700000,2470,WEST 1ST STREET,NaN,2022,...,59100,0,0,0,0,0,0,0,0,72
65996,Doc 2018092001289001,2018-09-20T00:00:00.000,3,7396,53,31900000,2355,EAST 12TH STREET,121,2022,...,114300,0,0,0,0,0,0,0,0,121
66682,Doc 2018083100413001,2018-08-29T00:00:00.000,3,6773,12,15000000,2626,KINGS HIGHWAY,48,2022,...,54700,0,0,0,0,0,0,0,0,48
68110,Doc 2018083100305001,2018-08-28T00:00:00.000,3,6801,56,27700000,1775,EAST 18TH STREET,NaN,2022,...,109312,0,0,0,0,0,0,0,0,96
73587,Doc 2018110101112001,2018-11-01T00:00:00.000,3,6716,75,9300000,1025,EAST 14TH STREET,NaN,2022,...,33600,0,0,0,0,0,0,0,0,35
77824,Doc 2018120300618001,2018-11-29T00:00:00.000,3,6803,51,15500000,2261,OCEAN AVENUE,54,2022,...,52812,0,0,0,0,0,0,0,0,54


### 4.D. A Function to Add the Target Data to the Comparables Table

In [114]:
target_property_df = pd.Series({"borough": target_borough,
                                         "block": target_block,
                                         "lot": target_lot})

target_property_df = target_property_df.append(pull_one_property_data(target_property_df))
target_property_df = target_property_df.to_frame().T

In [115]:
type(target_property_df)


comparables_df = target_property_df.append(comparables_df)
comparables_df = comparables_df[["document_id", "document_date", "borough", "block", "lot",
                                "document_amt", "street_number", "street_name", "unit", "year", "extracrdt", 
                                 "bldg_class", "owner", "num_bldgs", "yrbuilt", "yralt1", "yralt2", "period",
                                 "land_area", "bld_story", "gross_sqft", "lot_frt", "bld_frt", "lot_dep",
                                 "bld_dep", "office_area_gross", "residential_area_gross", "retail_area_gross", 
                                 "hotel_area_gross", "loft_area_gross", "factory_area_gross", "warehouse_area_gross",
                                 "storage_area_gross", "garage_area", "other_area_gross", "units"]]

In [116]:
comparables_df.to_csv('1669_East_19th_Street_apartment_building_comparable_sales_with_sqft_2016_to_now.csv', index = False)
# comparables_df[comparables_df["residential_area_gross"] != "0"].to_csv('59_west_70_street_townhouse_residential_comparable_sales_with_sqft_2019_to_now.csv', index = False)

# comparables_df.to_csv('comparable_sales_with_sqft_2019_to_now.csv', index = False)

In [117]:
comparables_df

,document_id,document_date,borough,block,lot,document_amt,street_number,street_name,unit,year,...,residential_area_gross,retail_area_gross,hotel_area_gross,loft_area_gross,factory_area_gross,warehouse_area_gross,storage_area_gross,garage_area,other_area_gross,units
0,NaN,NaN,3,6782,59,NaN,NaN,NaN,NaN,2022,...,0,0,0,0,0,0,0,0,0,29
2025,Doc 2017012600044001,2017-01-24T00:00:00.000,3,7397,34,37000000,1245,AVENUE X,135,2022,...,124636,0,0,0,0,0,0,0,0,136
3022,Doc 2017020900851001,2017-02-08T00:00:00.000,3,7380,34,13000000,2686,OCEAN AVENUE,NaN,2022,...,54000,0,0,0,0,0,0,0,0,48
5177,Doc 2017032100565001,2017-03-20T00:00:00.000,3,6768,17,3200000,1582,EAST 22ND STREET,NaN,2022,...,8125,0,0,0,0,0,0,0,0,8
5431,Doc 2017030100438001,2017-02-16T00:00:00.000,3,6541,46,4100000,1296,OCEAN PARKWAY,NaN,2022,...,9100,0,0,0,0,0,0,0,0,7
21220,Doc 2017071000392001,2017-06-27T00:00:00.000,3,8773,1134,1114000,1809,EMMONS AVENUE,PS75,2022,...,0,0,0,0,0,0,0,0,0,1
21221,Doc 2017071000392001,2017-06-27T00:00:00.000,3,8773,1024,1114000,1809,EMMONS AVENUE,5C,2022,...,0,0,0,0,0,0,0,0,0,1
28521,Doc 2017071000392001,2017-06-27T00:00:00.000,3,8773,1134,1114000,1809,EMMONS AVENUE,PS75,2022,...,0,0,0,0,0,0,0,0,0,1
28522,Doc 2017071000392001,2017-06-27T00:00:00.000,3,8773,1024,1114000,1809,EMMONS AVENUE,5C,2022,...,0,0,0,0,0,0,0,0,0,1
28820,Doc 2017091800596002,2017-09-14T00:00:00.000,3,6768,54,17550000,1695,EAST 21ST STREET,60,2022,...,76255,0,0,0,0,0,0,0,0,63
